In [ ]:
import pandas as pd

# Load data
train_df = pd.read_csv("/kaggle/input/telugu-lexicon/te.translit.sampled.train.tsv", sep="\t", header=None)
train_df.columns = ["target", "source", "label"]
#pairs = list(zip(train_df["source"], train_df["target"]))  # Latin to Devanagari

# Preview

val_df = pd.read_csv("/kaggle/input/telugu-lexicon/te.translit.sampled.dev.tsv", sep="\t", header=None)
val_df.columns = ["target", "source", "label"]
#val_pairs = list(zip(val_df["source"], val_df["target"]))

# Drop any rows where source or target is missing
train_df = train_df.dropna(subset=["source", "target"])
val_df = val_df.dropna(subset=["source", "target"])

# Ensure source and target are strings
train_df["source"] = train_df["source"].astype(str)
train_df["target"] = train_df["target"].astype(str)
val_df["source"] = val_df["source"].astype(str)
val_df["target"] = val_df["target"].astype(str)

# Create pairs
pairs = list(zip(train_df["source"], train_df["target"]))
val_pairs = list(zip(val_df["source"], val_df["target"]))

In [ ]:
print(len(pairs))

In [ ]:
import random

sample_size = 10000
pairs = random.sample(pairs, sample_size)    # use only 2000 training samples
#val_pairs = random.sample(val_pairs, sample_size)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


In [ ]:
#pairs = list(zip(df["source"], df["target"]))

#pairs = [
 #   ("namaste", "नमस्ते"),
  #  ("bharat",  "भारत"),
   # ("duniya",  "दुनिया"),
    #("prem",    "प्रेम"),
#]


In [ ]:
SRC_CHARS = set("".join(s for s, _ in pairs))
TRG_CHARS = set("".join(t for _, t in pairs)) | {"<sos>", "<eos>"}

src2idx = {ch: i+1 for i, ch in enumerate(sorted(SRC_CHARS))}  # reserve 0 for padding
src2idx["<pad>"] = 0
trg2idx = {ch: i+1 for i, ch in enumerate(sorted(TRG_CHARS))}
trg2idx["<pad>"] = 0

idx2trg = {i: ch for ch, i in trg2idx.items()}


In [ ]:
import torch
import torch.nn as nn

# === RNN Cell Wrapper === #
def get_rnn_cell(cell_type):
    cell_type = cell_type.upper()
    if cell_type == "GRU":
        return nn.GRU
    elif cell_type == "LSTM":
        return nn.LSTM
    elif cell_type == "RNN":
        return nn.RNN
    else:
        raise ValueError("Unsupported RNN cell type. Use 'RNN', 'GRU', or 'LSTM'.")

# === Encoder === #
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, cell_type="GRU", dropout=0.0):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=0)
        self.rnn = get_rnn_cell(cell_type)(emb_dim, hidden_dim, batch_first=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))  # [src_len, 1, emb_dim]
        outputs, hidden = self.rnn(embedded)  # outputs: [src_len, 1, hidden_dim]
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))

    def forward(self, hidden, encoder_outputs):
        # Handle LSTM or GRU hidden state
        if isinstance(hidden, tuple):  # LSTM: (h, c)
            hidden = hidden[0]  # use only h_n for attention

        src_len = encoder_outputs.size(0)
        hidden = hidden.repeat(src_len, 1, 1)  # [src_len, batch, hidden_dim]

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [src_len, batch, hidden_dim]
        energy = energy.squeeze(1)  # [src_len, hidden_dim]
        v = self.v.repeat(src_len, 1)
        scores = torch.sum(v * energy, dim=1)  # [src_len]
        return torch.softmax(scores, dim=0).unsqueeze(1)  # [src_len, 1]

class AttnDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, cell_type="GRU", dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=0)
        self.rnn = get_rnn_cell(cell_type)(emb_dim + hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)
        self.attention = Attention(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_step, hidden, encoder_outputs):
        # input_step: [1, 1], hidden: [1, 1, hidden_dim], encoder_outputs: [src_len, 1, hidden_dim]
        embedded = self.dropout(self.embedding(input_step))  # [1, 1, emb_dim]

        # Compute attention weights and context
        attn_weights = self.attention(hidden, encoder_outputs)  # [src_len, 1]
        attn_weights = attn_weights.permute(1, 0)  # [1, src_len]

        # encoder_outputs: [src_len, 1, hidden_dim] → [1, src_len, hidden_dim]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        # context: [1, 1, hidden_dim]
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)  # [1, 1, hidden_dim]

        # Combine embedding and context
        rnn_input = torch.cat((embedded, context), dim=2)  # [1, 1, emb_dim + hidden_dim]
        output, hidden = self.rnn(rnn_input, hidden)  # same shape
        output = output.squeeze(0)
        context = context.squeeze(0)
        pred = self.fc_out(torch.cat((output, context), dim=1))  # [1, output_dim]
        return pred, hidden



# === Decoder === #
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, cell_type="GRU", num_layers=1, dropout=0.0):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=0)
        self.rnn = get_rnn_cell(cell_type)(
            emb_dim, hidden_dim, num_layers, dropout=dropout if num_layers > 1 else 0.0
        )
        self.out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_step, hidden):
        embedded = self.embedding(input_step)  # [1, 1, emb_dim]
        output, hidden = self.rnn(embedded, hidden)
        output = self.dropout(output.squeeze(0))  # Apply dropout to RNN output
        prediction = self.out(output)             # [1, output_dim]
        return prediction, hidden

# === Seq2Seq Wrapper === #
class AttnSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, sos_idx):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.sos_idx = sos_idx

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        output_dim = self.decoder.fc_out.out_features
        outputs = torch.zeros(trg_len, 1, output_dim).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        input_step = torch.tensor([[self.sos_idx]], device=self.device)

        for t in range(trg_len):
            output, hidden = self.decoder(input_step, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1).unsqueeze(0)
            input_step = trg[t].unsqueeze(0) if teacher_force else top1

        return outputs

    def adjust_hidden_for_decoder(self, hidden, target_layers):
        """
        Adjust the encoder's hidden state to match the number of decoder layers.
        Pads if encoder has fewer layers, trims if more.
        Works for GRU/RNN (tensor) and LSTM (tuple).
        """
        if isinstance(hidden, tuple):  # LSTM
            h, c = hidden
            h = self._match_layers(h, target_layers)
            c = self._match_layers(c, target_layers)
            return (h, c)
        else:  # GRU or RNN
            return self._match_layers(hidden, target_layers)

    def _match_layers(self, state, target_layers):
        """
        Pad or trim the hidden state tensor to match target number of layers.
        """
        current_layers = state.size(0)
        if current_layers == target_layers:
            return state
        elif current_layers < target_layers:
            diff = target_layers - current_layers
            pad = torch.zeros(diff, state.size(1), state.size(2), device=state.device)
            return torch.cat([state, pad], dim=0)
        else:  # current_layers > target_layers
            return state[:target_layers]


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
'''
import torch.optim as optim

encoder = Encoder(len(src2idx), emb_dim, hidden_dim, cell_type, num_layers)
decoder = Decoder(len(trg2idx), emb_dim, hidden_dim, cell_type, num_layers)
sos_idx = trg2idx["<sos>"]
model = Seq2Seq(encoder, decoder, DEVICE, sos_idx).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=trg2idx["<pad>"])
'''
def tensor_from_word(word, mapping, add_eos=False):
    indices = [mapping[ch] for ch in word]
    if add_eos:
        indices.append(mapping["<eos>"])
    return torch.tensor(indices, dtype=torch.long, device=DEVICE).unsqueeze(1)

'''
for epoch in range(1, 6):
    model.train()
    epoch_loss = 0
    train_correct = 0
    train_total = 0

    for src_word, trg_word in pairs:
        src_tensor = tensor_from_word(src_word, src2idx)
        trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)

        optimizer.zero_grad()
        output = model(src_tensor, trg_tensor)
        output_dim = output.shape[-1]
        loss = criterion(output.view(-1, output_dim), trg_tensor.view(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # Training accuracy
        pred_tokens = output.argmax(-1).view(-1)
        true_tokens = trg_tensor.view(-1)
        mask = true_tokens != trg2idx["<pad>"]
        correct = (pred_tokens == true_tokens) & mask
        train_correct += correct.sum().item()
        train_total += mask.sum().item()

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for src_word, trg_word in val_pairs:
            src_tensor = tensor_from_word(src_word, src2idx)
            trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)
            output = model(src_tensor, trg_tensor)
            loss = criterion(output.view(-1, output.shape[-1]), trg_tensor.view(-1))
            val_loss += loss.item()

            pred_tokens = output.argmax(-1).view(-1)
            true_tokens = trg_tensor.view(-1)
            mask = true_tokens != trg2idx["<pad>"]
            correct = (pred_tokens == true_tokens) & mask
            val_correct += correct.sum().item()
            val_total += mask.sum().item()

    train_acc = train_correct / train_total
    val_acc = val_correct / val_total
    print(f"Epoch {epoch:3d} | Train Loss: {epoch_loss/len(pairs):.4f} | "
          f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_pairs):.4f} | Val Acc: {val_acc:.4f}")

'''

In [ ]:
def beam_search(model, src_tensor, beam_size=3, max_len=30):
    model.eval()
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

        if isinstance(hidden, tuple):
            h, c = hidden
            hidden = (
                h[:model.decoder.rnn.num_layers],
                c[:model.decoder.rnn.num_layers],
            )
        else:
            hidden = hidden[:model.decoder.rnn.num_layers]

        sequences = [([model.sos_idx], 0.0, hidden)]

        for _ in range(max_len):
            all_candidates = []
            for seq, score, h in sequences:
                input_step = torch.tensor([[seq[-1]]], device=model.device)
                output, h_new = model.decoder(input_step, h, encoder_outputs)
                probs = torch.log_softmax(output, dim=1)
                topk = torch.topk(probs, beam_size)

                for i in range(beam_size):
                    token = topk.indices[0][i].item()
                    token_score = topk.values[0][i].item()
                    new_seq = seq + [token]
                    all_candidates.append((new_seq, score + token_score, h_new))

            sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_size]

            if all(seq[-1] == trg2idx["<eos>"] for seq, _, _ in sequences):
                break

        best_seq = sequences[0][0][1:]  # remove <sos>
        return "".join([idx2trg[i] for i in best_seq if i != trg2idx["<eos>"]])


In [ ]:
!pip install -q wandb
import wandb


In [ ]:
wandb.login(key='af7d7cf29d8954a13afb06c7a0d0c196c36ac51b')


In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "emb_dim": {"values": [16,32,64,256]},
        "hidden_dim": {"values": [16,32,64,256]},
        "cell_type": {"values": ["RNN","GRU","LSTM"]},
        "enc_layers": {"values": [1]},
        "dec_layers": {"values": [1]},
        "dropout": {"values": [0,0.2, 0.3]},
        "beam_size": {"values": [1, 3, 2]},
        "lr": {"values": [0.001, 0.0005]}
    }
}


In [ ]:
def train_model(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        run.name = f"emb{config.emb_dim}_hid{config.hidden_dim}_{config.cell_type}_enc{config.enc_layers}_dec{config.dec_layers}_drop{int(config.dropout*100)}_beam{config.beam_size}_lr{config.lr}"

        encoder = Encoder(len(src2idx), config.emb_dim, config.hidden_dim, cell_type=config.cell_type, dropout=config.dropout)
        decoder = AttnDecoder(len(trg2idx), config.emb_dim, config.hidden_dim, cell_type=config.cell_type, dropout=config.dropout)

        model = AttnSeq2Seq(encoder, decoder, DEVICE, sos_idx=trg2idx["<sos>"]).to(DEVICE)

        optimizer = optim.Adam(model.parameters(), lr=config.lr)
        criterion = nn.CrossEntropyLoss(ignore_index=trg2idx["<pad>"])
        beam_size = config.get("beam_size", 3)

        for epoch in range(1, 6):
            model.train()
            total_loss = 0
            train_correct = 0
            train_total = 0

            for src_word, trg_word in pairs:
                src_tensor = tensor_from_word(src_word, src2idx)
                trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)

                optimizer.zero_grad()
                output = model(src_tensor, trg_tensor, teacher_forcing_ratio=config.get("teacher_forcing_ratio", 0.5))
                loss = criterion(output.view(-1, output.size(-1)), trg_tensor.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

                pred_tokens = output.argmax(-1).view(-1)
                true_tokens = trg_tensor.view(-1)
                mask = true_tokens != trg2idx["<pad>"]
                correct = (pred_tokens == true_tokens) & mask
                train_correct += correct.sum().item()
                train_total += mask.sum().item()

            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            exact_match_count = 0

            with torch.no_grad():
                for src_word, trg_word in val_pairs:
                    src_tensor = tensor_from_word(src_word, src2idx)
                    trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)

                    output = model(src_tensor, trg_tensor, teacher_forcing_ratio=0.0)
                    loss = criterion(output.view(-1, output.size(-1)), trg_tensor.view(-1))
                    val_loss += loss.item()

                    pred_tokens = output.argmax(-1).view(-1)
                    true_tokens = trg_tensor.view(-1)
                    mask = true_tokens != trg2idx["<pad>"]
                    correct = (pred_tokens == true_tokens) & mask
                    val_correct += correct.sum().item()
                    val_total += mask.sum().item()

                    # Beam search for exact match accuracy
                    pred_str = beam_search(model, src_tensor, beam_size=beam_size)
                    if pred_str == trg_word:
                        exact_match_count += 1

            train_acc = train_correct / train_total
            val_acc = val_correct / val_total
            exact_match = exact_match_count / len(val_pairs)

            print(f"Epoch {epoch:2d} | "
                  f"Train Loss: {total_loss / len(pairs):.4f} | "
                  f"Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss / len(val_pairs):.4f} | "
                  f"Val Acc: {val_acc:.4f} | "
                  f"Val Exact Match: {exact_match:.4f} | "
                  f"Beam Size: {beam_size}")


            wandb.log({
                "epoch": epoch,
                "train_loss": total_loss / len(pairs),
                "val_loss": val_loss / len(val_pairs),
                "train_accuracy": train_correct / train_total,
                "val_accuracy": val_correct / val_total,
                "val_exact_match": exact_match_count / len(val_pairs),
                "beam_size": beam_size
            })


In [ ]:
'''
sweep_id = wandb.sweep(sweep_config, project="transliteration-with-attn-sweep")
wandb.agent(sweep_id, function=train_model, count=50)
wandb.finish()
'''

In [ ]:
best_config = {
    "emb_dim": 16,
    "hidden_dim": 256,
    "cell_type": "LSTM",
    "enc_layers": 1,
    "dec_layers": 1,
    "dropout": 0,
    "beam_size": 1,
    "lr": 0.0005}


In [ ]:
def best_train_model(config=None):
    with wandb.init(config=config):
        config = wandb.config

        encoder = Encoder(len(src2idx), config.emb_dim, config.hidden_dim,
                          config.cell_type, config.dropout)
        decoder = AttnDecoder(len(trg2idx), config.emb_dim, config.hidden_dim,
                      config.cell_type, dropout=config.dropout)
        model = AttnSeq2Seq(encoder, decoder, DEVICE, sos_idx=trg2idx["<sos>"]).to(DEVICE)

        optimizer = optim.Adam(model.parameters(), lr=config.lr)
        criterion = nn.CrossEntropyLoss(ignore_index=trg2idx["<pad>"])
        beam_size = config.get("beam_size", 3)

        for epoch in range(1, 6):
            model.train()
            total_loss = 0
            train_correct = 0
            train_total = 0

            for src_word, trg_word in pairs:
                src_tensor = tensor_from_word(src_word, src2idx)
                trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)

                optimizer.zero_grad()
                output = model(src_tensor, trg_tensor, teacher_forcing_ratio=config.get("teacher_forcing_ratio", 0.5))
                loss = criterion(output.view(-1, output.size(-1)), trg_tensor.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

                pred_tokens = output.argmax(-1).view(-1)
                true_tokens = trg_tensor.view(-1)
                mask = true_tokens != trg2idx["<pad>"]
                correct = (pred_tokens == true_tokens) & mask
                train_correct += correct.sum().item()
                train_total += mask.sum().item()

            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            exact_match_count = 0

            with torch.no_grad():
                for src_word, trg_word in val_pairs:
                    src_tensor = tensor_from_word(src_word, src2idx)
                    trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)

                    output = model(src_tensor, trg_tensor, teacher_forcing_ratio=0.0)
                    loss = criterion(output.view(-1, output.size(-1)), trg_tensor.view(-1))
                    val_loss += loss.item()

                    pred_tokens = output.argmax(-1).view(-1)
                    true_tokens = trg_tensor.view(-1)
                    mask = true_tokens != trg2idx["<pad>"]
                    correct = (pred_tokens == true_tokens) & mask
                    val_correct += correct.sum().item()
                    val_total += mask.sum().item()

                    # Beam search for exact match accuracy
                    pred_str = beam_search(model, src_tensor, beam_size=beam_size)
                    if pred_str == trg_word:
                        exact_match_count += 1

            train_acc = train_correct / train_total
            val_acc = val_correct / val_total
            exact_match = exact_match_count / len(val_pairs)

            print(f"Epoch {epoch:2d} | "
                  f"Train Loss: {total_loss / len(pairs):.4f} | "
                  f"Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss / len(val_pairs):.4f} | "
                  f"Val Acc: {val_acc:.4f} | "
                  f"Val Exact Match: {exact_match:.4f} | "
                  f"Beam Size: {beam_size}")

            wandb.log({
                "epoch": epoch,
                "train_loss": total_loss / len(pairs),
                "val_loss": val_loss / len(val_pairs),
                "train_accuracy": train_acc,
                "val_accuracy": val_acc,
                "val_exact_match": exact_match,
                "beam_size": beam_size
            })

        return model


In [ ]:
model = best_train_model(config=best_config)


In [ ]:

test_df = pd.read_csv("/kaggle/input/telugu-lexicon/te.translit.sampled.test.tsv", sep="\t", header=None)
test_df.columns = ["target", "source", "label"]  # adjust if only 2 columns
test_df = test_df.dropna(subset=["source", "target"])
test_df["source"] = test_df["source"].astype(str)
test_df["target"] = test_df["target"].astype(str)
test_pairs = list(zip(test_df["source"], test_df["target"]))


In [ ]:
print(len(test_pairs))

In [ ]:
model.eval()
test_correct = 0
test_total = 0
exact_match_count = 0
beam_size = best_config["beam_size"]

with torch.no_grad():
    for src_word, trg_word in test_pairs:
        src_tensor = tensor_from_word(src_word, src2idx)
        trg_tensor = tensor_from_word(trg_word, trg2idx, add_eos=True)

        output = model(src_tensor, trg_tensor, teacher_forcing_ratio=0.0)

        # Character-level accuracy
        pred_tokens = output.argmax(-1).view(-1)
        true_tokens = trg_tensor.view(-1)
        mask = true_tokens != trg2idx["<pad>"]
        correct = (pred_tokens == true_tokens) & mask
        test_correct += correct.sum().item()
        test_total += mask.sum().item()

        # Word-level exact match via beam search
        pred_str = beam_search(model, src_tensor, beam_size=beam_size)
        if pred_str == trg_word:
            exact_match_count += 1

test_char_acc = test_correct / test_total
test_exact_match = exact_match_count / len(test_pairs)

print(f"Test Char Accuracy: {test_char_acc:.4f}")
print(f"Test Exact Match:  {test_exact_match:.4f}")


In [ ]:
from tabulate import tabulate

def display_predictions(model, test_pairs, beam_size=3, max_samples=10):
    model.eval()
    table = []

    with torch.no_grad():
        sampled_pairs = random.sample(test_pairs, k=min(max_samples, len(test_pairs)))
        for src_word, trg_word in sampled_pairs:
            src_tensor = tensor_from_word(src_word, src2idx)
            pred_str = beam_search(model, src_tensor, beam_size=beam_size)
            match = "✅" if pred_str == trg_word else "❌"
            table.append([src_word, pred_str, trg_word, match])

    headers = ["Input (Latin)", "Predicted (Telugu)", "Target (Telugu)", "Match"]
    print(tabulate(table, headers=headers, tablefmt="fancy_grid"))


In [ ]:
display_predictions(model, test_pairs, beam_size=best_config["beam_size"], max_samples=10)


In [ ]:
import csv, os, pandas as pd, torch

# --- 1. Run model on the whole test set once ---
model.eval()
beam = best_config["beam_size"]
records = []                         # rows for the output file

with torch.no_grad():
    for latin, true in test_pairs:
        pred = beam_search(model,
                           tensor_from_word(latin, src2idx),
                           beam_size=beam)
        records.append({"latin": latin,
                        "true": true,
                        "pred": pred})

# --- 2. Save as TSV (or CSV) ---
out_df = pd.DataFrame(records)
os.makedirs("predictions_attention", exist_ok=True)          # local folder
out_path = "predictions_attention/test_predictions.tsv"
out_df.to_csv(out_path, sep="\t", index=False)
print(f"Saved {len(out_df)} rows ➜ {out_path}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np

# path after you upload
font_path = "/kaggle/input/nototsanstelugu/NotoSansTelugu-Regular.ttf"
fm.fontManager.addfont(font_path)
plt.rcParams["font.family"] = "Noto Sans Telugu"

In [ ]:
def decode_with_attention(model, src_word, src2idx, trg2idx, idx2trg, max_len=30):
    model.eval()
    with torch.no_grad():
        src_tensor = tensor_from_word(src_word, src2idx)
        encoder_outputs, hidden = model.encoder(src_tensor)
        input_step = torch.tensor([[trg2idx["<sos>"]]], device=DEVICE)

        output_tokens = []
        attention_weights = []

        for _ in range(max_len):
            # manually invoke attention
            attn_weights = model.decoder.attention(hidden[0] if isinstance(hidden, tuple) else hidden, encoder_outputs)
            attn_weights = attn_weights.permute(1, 0)  # [1, src_len]
            context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs.permute(1, 0, 2))  # [1, 1, hidden_dim]
            embedded = model.decoder.embedding(input_step)
            rnn_input = torch.cat((embedded, context), dim=2)

            output, hidden = model.decoder.rnn(rnn_input, hidden)
            output = output.squeeze(0)
            context = context.squeeze(0)
            pred = model.decoder.fc_out(torch.cat((output, context), dim=1))  # [1, output_dim]

            top1 = pred.argmax(1).item()
            output_tokens.append(top1)
            attention_weights.append(attn_weights.squeeze(0).cpu().numpy())

            if idx2trg[top1] == "<eos>":
                break

            input_step = torch.tensor([[top1]], device=DEVICE)

        return output_tokens, np.stack(attention_weights)  # [output_len], [output_len x src_len]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# -------------------------------------------------------------------
# helper  A · grid of attention maps
# -------------------------------------------------------------------
import matplotlib.pyplot as plt
import numpy as np
import random

def plot_attention_grid(model, test_pairs, src2idx, trg2idx, idx2src, idx2trg, max_samples=9, max_len=30):

    """
    Plots a 3x3 grid of attention heatmaps for randomly sampled test pairs.
    """
    # --- always sample safely ---
    sampled = random.sample(test_pairs, min(max_samples, len(test_pairs)))

    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    fig.suptitle("Attention Heatmaps", fontsize=16)

    for ax, (src_word, trg_word) in zip(axes.flat, sampled):
        pred_idx, attn = decode_with_attention(model, src_word,
                                               src2idx, trg2idx, idx2trg,
                                               max_len=max_len)

        src_labels = list(src_word)
        trg_labels = [idx2trg[i] for i in pred_idx]

        im = ax.imshow(attn, cmap="viridis", aspect="auto")
        ax.set_xticks(np.arange(len(src_labels)))
        ax.set_yticks(np.arange(len(trg_labels)))
        ax.set_xticklabels(src_labels, rotation=45, ha="right")
        ax.set_yticklabels(trg_labels)
        ax.set_xlabel("Input")
        ax.set_ylabel("Output")
        ax.set_title(f'"{src_word}" → "{trg_word}"')

    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------------
# helper  B · single attention map
# -------------------------------------------------------------------
def plot_attention_map(attn, src_word, pred_tokens, idx2trg):
    src_labels = list(src_word)
    trg_labels = [idx2trg[i] for i in pred_tokens]

    plt.figure(figsize=(8, 6))
    im = plt.imshow(attn, cmap="viridis")
    plt.xticks(np.arange(len(src_labels)), src_labels, rotation=45, ha="right")
    plt.yticks(np.arange(len(trg_labels)), trg_labels)
    plt.xlabel("Input Sequence")
    plt.ylabel("Decoded Output")
    plt.title("Attention Heat-map")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()


In [ ]:
idx2src = {i: ch for ch, i in src2idx.items()}
idx2trg = {i: ch for ch, i in trg2idx.items()}


plot_attention_grid(model, test_pairs, src2idx, trg2idx, idx2src, idx2trg, max_samples=9)


In [ ]:
pred_tokens, attn_matrix = decode_with_attention(model, src_word, src2idx, trg2idx, idx2trg)


In [ ]:
import seaborn as sns

def plot_attention_map(attn, src_word, pred_tokens, idx2trg):
    src_labels = list(src_word)
    trg_labels = [idx2trg[i] for i in pred_tokens]

    plt.figure(figsize=(10, 8))
    sns.heatmap(attn, xticklabels=src_labels, yticklabels=trg_labels, cmap="viridis", cbar=True)
    plt.xlabel("Input Sequence")
    plt.ylabel("Decoded Output")
    plt.title("Attention Heatmap")
    plt.show()


In [ ]:
plot_attention_map(attn_matrix, src_word, pred_tokens, idx2trg)


In [ ]:
pip install -U kaleido

In [ ]:
from ipywidgets import interact, IntSlider, Dropdown, HBox, VBox
from IPython.display import HTML, display
import numpy as np

def attention_blocks_html(src, pred, attn, step):
    """
    Renders input chars in a row of colored blocks for one output‐step.
    
    src  : str, input sequence
    pred : str, predicted output sequence
    attn : np.array shape (len(pred), len(src))
    step : int, which output index to show
    """
    # clamp step
    step = min(max(step, 0), attn.shape[0] - 1)
    weights = attn[step]  # shape = [len(src)]
    
    # Build HTML
    html = "<div style='font-family: monospace;'>"
    html += f"<strong>Decoding step {step} → “{pred[step]}”</strong></div>"
    html += "<table style='border-collapse: collapse;'><tr>"
    for ch, w in zip(src, weights):
        # use red→transparent, alpha = w
        bg = f"rgba(255,0,0,{w:.3f})"
        html += (
            f"<td "
            f"style='border:1px solid #aaa; background:{bg}; "
            f"padding:8px; font-size:20px; text-align:center;'>"
            f"{ch}</td>"
        )
    html += "</tr></table>"
    
    display(HTML(html))



In [ ]:
from ipywidgets import Dropdown, IntSlider, HBox, VBox, Output
from IPython.display import display

def interactive_attention_blocks(cache):
    """
    cache : list of (src, pred, attn) tuples
    """
    # 1) Dropdown for selecting the example
    ex_opts = {f"{i}: {s}→{p}": i for i,(s,p,_) in enumerate(cache)}
    ex_dd  = Dropdown(options=ex_opts, description="Example")

    # 2) A slider, dummy-initialized
    slider = IntSlider(description="Step", min=0, max=0, value=0)

    # 3) An output area
    out = Output()

    # Callback #1: when you pick a new example, update slider.max to len(pred)-1
    def _on_example_change(change):
        idx = change['new']
        _, pred, _ = cache[idx]
        slider.max = len(pred) - 1
        slider.value = 0           # reset to first step

    ex_dd.observe(_on_example_change, names='value')

    # Callback #2: whenever example *or* slider changes, redraw
    def _update(_):
        with out:
            out.clear_output(wait=True)
            idx  = ex_dd.value
            step = slider.value
            src, pred, attn = cache[idx]
            # clamp step just in case
            step = max(0, min(step, len(pred)-1))
            attention_blocks_html(src, pred, attn, step)

    ex_dd.observe(_update, names='value')
    slider.observe(_update, names='value')

    # 4) Initialize slider range & draw initial view
    _on_example_change({'new': ex_dd.value})
    _update(None)

    # 5) Put it all together
    controls = HBox([ex_dd, slider])
    widget = VBox([controls, out])
    display(widget)


In [ ]:
# 1. Build a small cache
import random
cache = []
for src, trg in random.sample(test_pairs, k=min(9, len(test_pairs))):
    pred_idx, attn = decode_with_attention(model, src, src2idx, trg2idx, idx2trg)
    pred = "".join(idx2trg[i] for i in pred_idx if idx2trg[i] != "<eos>")
    cache.append((src, pred, attn))

# 2. Call the widget
interactive_attention_blocks(cache)
